# BC205: Algorithms for Bioinformatics.

### Exercise 3. Implementing a simplified version of the FastA algorithm

Guide for the constructed FastA algorithm:

1) a simple score of +1/0 for word matches/mismatches. This means that S[i] will be incremmented by +1 each time a k-mer is found.
2) k=5 as the length of the matched word
3) m=20 as the minimum size of matched diagonals (S[i]) that you will keep in each search
4) g=3 as the maximum gap size before joining regions
5) a total score of 50% of the length of the query sequence to be used as prerequisite
6) Final outputs will consist of a list of matched sequences that fulfill the above criteria alongside with their Total Scores.
7) Report the highest matching sequence. 

- **Download the data**

In [13]:
import os
import requests


# create the data directory if it doesn't exist
data_dir = "./data" # define the path for the data directory
if not os.path.exists(data_dir): # check if the directory already exists
    os.makedirs(data_dir) # create the directory if it doesn't exist


# dictionary with file name and URL
file_to_download = {"query.fa": "https://raw.githubusercontent.com/christoforos-nikolaou/BC205/master/files/query.fa",
                    "all_yeast_genes_minplus1k.fa": "https://dl.dropboxusercontent.com/scl/fi/85mp7f5iwirkjtjd3vwcg/all_yeast_genes_minplus1k.fa?rlkey=i4etsyv8zmia2wnagm07nym51&st=am73dxei"}


# download files
for filename, url in file_to_download.items():
    file_path = os.path.join(data_dir, filename) # create the full file path
    if not os.path.exists(file_path): # check if the file already exists and if not download it
        print(f"Downloading {filename} ...")
        response = requests.get(url) # using requests to download the file when needed
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {filename} successfully.")
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")        
    else:
        print(f"{filename} already exists. Skipping download.") # if the file already exists, skip the downloading




query.fa already exists. Skipping download.
all_yeast_genes_minplus1k.fa already exists. Skipping download.


In [17]:
query = ""
with open ('data/query.fa') as f:
    for i in f.readlines():
        if i.startswith('>'):
            continue
        query += i.strip()
print(query)

GTATATAACCTAAAAAGAAAATTTGATTAACGAATTCAGAACCCACACGCTGAGACAAGGGGCCCCTGATTATACACCTCTGGAAATGGAAATGGTTTGAATGGGCAAAGCCAGGAAGATATTGAGATTTTAACATATATATTATGTGACATTTGCCCTTCTTTTAGATGGTAGATAATGCTTATTTATGTAGCACGGGTCATTCATCATGGATTTGCGGAGTGGAAGACCAAATGATCATTAACCTGCTACCTGCTTTCAGAATCCTAATGATTTCAAAATGGATAACTTTACCTGTCTCTCAAATTCAGGTCTATTATCTCTCCACAAGATGCAAGCATCAATGTTGGCACCACTTTCGATATTGGGCTCACTCAACATGCTCATAACACTTAATAGAATTTTTTCTACACTTTGCACTGGCGACCATCTTTCTTCCGCTAATTCGTACATGTTAGGATCATCACCAGGGGAGTGTAGAATGGATATGCACACTTCCCCATTTGGATAAATATTTGGATGTAGTATGCTGGGTGTGAAAGTAAGTTTAGGTGGAGATAACGGATAGTCTTTAGGAAACTCTAGCTTAGCATTAAAAACACCATCAGCGTATGGCGTATCTGGAGGCCCTTGAATTAGGCAGTCCCAAATGAATATGTTATTCTCCGATTTGGGACCAGCCACTATACCAGGTGGAGAATCTTTAATTAACTGTTGAAGCTCCTTGAGGAGACGTTTCTGAGCGGTTTTCGACATGCTATGCCCTTCCAAATTACACTATTACTAGGGAAGTTCCTTTTAGGATAATCTCCTTCGTACGCTAAACGCCAGAGTTTACTTTGGCGCTTTTCGAGCTCTTGGTTTAGAGGCTGTAATCTCGTTTTCGGGTAATGGCGAAAAGGAGTTATAAGAAGGATCTCGAGACAATAAGCTGCTGCATCTTCGTGAGGTAGATGCGATGAGGCGCCTTGTTTTAAACTTGAAACAGTCTGGTAAGTTC

In [ ]:
from collections import defaultdict
from Bio import SeqIO

# Parameters
k = 5       # k-mer length
m = 20      # min diagonal score
g = 3       # max gap to join diagonals
threshold_percent = 0.5

# Step 1: Build k-mer index from query
## The algorithm first looks for short, exact matches between the query sequence and the target sequences.

def build_kmer_index(query, k):
    kmer_index = defaultdict(list) ## Creates a dictionary where each key will be a k-mer, and its value will be a list of starting positions where that k-mer appears in the query sequence.
    for i in range(len(query) - k + 1): ## extract all possible k-mers
        kmer = query[i:i + k] ## Extracts the k-mer starting at position i
        kmer_index[kmer].append(i) ## Adds the starting position i to the list associated with the extracted kmer
    return kmer_index

## a dictionary with keys all the possible k-mers and values the starting positions of these k-mers in the query sequence



## When a 5-mer from the target sequence (let's say it starts at position j) matches a 5-mer from the query sequence (let's say it starts at position i), 
## this match implies a potential alignment.

## The "diagonal" for this match is j - i = 1 - 3 = -2.

## The value associated with that key (in S dictionary) is the count of k-mer matches found on that particular diagonal.
## when a diagonal key in S is a high count it means that many k-mer matches fall on that specific diagonal. (dense concentration of short, exact matches)


## The crucial point is that FastA doesn't initially try to distinguish between individual alignments for a given diagonal. 
## Instead, it treats all k-mer matches that fall on the same diagonal as evidence for a single, potentially continuous, ungapped alignment.
## It's about identifying "regions of high density" of matches, not pinpointing exact individual alignments at this stage.



# Step 2: Score diagonals for a target sequence
def score_diagonals(target, kmer_index, k):
    S = defaultdict(int)
    for j in range(len(target) - k + 1):
        kmer = target[j:j + k]
        if kmer in kmer_index: ## checks if the k-mer exists in the query sequence
            for i in kmer_index[kmer]: ## iterates through all the starting positions (i) where that k-mer appeared in the query
                diag = j - i ## represents the offset between the start position of the k-mer in the target sequence (j) and the start position in the query sequence (i). 
                             ## Consistent diagonals indicate regions of similarity.
                S[diag] += 1
    return S




## The filter_and_join_diagonals step then reinforces this. It takes these individual diagonal scores and 
## looks for sequences of strong diagonals that are close to each other (within the g gap parameter). 
## This process implicitly "builds" a larger, more comprehensive alignment region.


# Step 3: Filter and join diagonals
def filter_and_join_diagonals(S, m, g): ## significant regions of similarity might be broken by gaps. FastA attempts to join nearby "good" diagonals if the gap between them is small.
    diagonals = sorted([d for d in S if S[d] >= m]) ## This line filters the S dictionary to keep only those diagonals whose scores are greater than or equal to m. It then sorts these filtered diagonals.
    regions = []
    
    if not diagonals:
        return ('No match')
    
    current_region = [diagonals[0]] ## Start with the first diagonal in the sorted list
    for d in diagonals[1:]: ## Iterates through the sorted diagonals, starting from the second one.
        if d - current_region[-1] <= g: ## It checks if the current diagonal d is "close enough" to the last diagonal in current_region
            current_region.append(d)
        else: ## If the gap is too large, it means the current diagonal starts a new potential region
            # Score this region
            region_score = sum(S[x] for x in current_region) ## This line calculates the total score for the current region by summing the scores of all diagonals in current_region.
            regions.append(region_score)
            current_region = [d] ## Start a new region with the current diagonal d (which is not close enough to the previous one)
    
    # Add the last region
    region_score = sum(S[x] for x in current_region)
    regions.append(region_score)
    
    return max(regions) if regions else 'No regions found' ## Returns the maximum score among all regions found, or a message if no regions were found.



# Step 4: Run search over all target genes
def search_fasta(query, fasta_file):
    kmer_index = build_kmer_index(query, k) ## Builds the k-mer index for your query sequence once
    results = [] ## Initializes an empty list to store the matched gene IDs and their scores.
    required_score = int(len(query) * threshold_percent) ## Calculates the required score (to be considered a significant match) based on the length of the query sequence and the threshold percentage.
    
    for record in SeqIO.parse(fasta_file, "fasta"): ## This loop iterates through each sequence record in your FASTA file. 
        gene_id = record.id ## Extracts the gene ID from the record.
        sequence = str(record.seq) ## Converts the sequence to a string format.
        S = score_diagonals(sequence, kmer_index, k) ## Calls score_diagonals to get the scores for all diagonals for the current target sequence.
        total_score = filter_and_join_diagonals(S, m, g) ## Calls filter_and_join_diagonals to find the highest scoring joined region for the current target.
        
        if total_score >= required_score: ## Checks if the total_score for this target sequence meets the defined threshold.
            results.append((gene_id, total_score))
    
    # Sort by total_score
    results.sort(key=lambda x: x[1], reverse=True) ## Sorts the results list (gene names) in descending order based on the total_score
    return results


In [21]:
for record in SeqIO.parse('data/all_yeast_genes_minplus1k.fa', "fasta"):
    gene_id = record.id
    sequence = str(record.seq)
    #
    # print(sequence)

- Use the most common k-mers instead of the full k-mers list:

For example, you could count the occurrences of each k-mer in the query and only include those k-mers that appear above a certain frequency in kmer_index

In [ ]:
# Inside build_kmer_index, after building kmer_index initially:
# You could add a step to filter based on frequency, e.g.:
# from collections import Counter
# kmer_counts = Counter(query[i:i+k] for i in range(len(query) - k + 1))
# frequent_kmers = {kmer for kmer, count in kmer_counts.items() if count > some_threshold}
#
# Then modify score_diagonals to check 'if kmer in frequent_kmers' first,
# or rebuild kmer_index to only contain frequent k-mers.

This is a **trade-off**. While it can speed up the process, it might cause you to miss some legitimate matches if the significant k-mers are not among the "most common" ones.

- Increase the size of k:

When k is larger, the probability of a random k-mer match decreases significantly. This means that when you do find a match, it's more likely to be biologically meaningful and less likely to be by chance. This reduces the number of "spurious" (false positive) k-mer matches that score_diagonals has to process.

A larger k leads to a smaller number of unique k-mers (for a given sequence length), as each k-mer is longer and less likely to repeat frequently. This can result in a smaller kmer_index, which can be faster to build and search.

**trade-off** : While increasing k makes the search more specific (fewer false positives), it also makes it less sensitive. You might miss matches where the sequence similarity is lower or involves more insertions/deletions, as finding long exact k-mer matches becomes harder.